# Data Analysis Notebook
## Basic idea is this:
1. Load data you are interested into a dataframe
2. Pass dataframe to analysis function, which produces plot or whatever

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from plotly import __version__
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
from plotly import tools
import calendar
from neighbourhoods import hoods, cities
import colorlover as cl
import tabulate
import datetime
from IPython.display import HTML, display
import matplotlib.path as mplPath
from sklearn import datasets, linear_model
import math
#import tabulate
init_notebook_mode()
conn = sqlite3.connect('apartments.db')
c = conn.cursor()

In [ ]:
def generate_months(startdate=datetime.date(2017,1,1),enddate=datetime.date.today()):
    month = startdate.month
    year = startdate.year
    months = []
    i = 0 
    while (year < enddate.year) or (month <= enddate.month):
        months.append(str(datetime.date(year,month,1))[:7])
        if (month % 12) == 0:
            year = year + 1
            month = 0
        month = month + 1
    return months

## Rental Report:
# First, set the min and max prices to eliminate outliers
min_price = 500
max_price = 7000

# Months to plot
months = generate_months()
#months.remove('2018-01')

In [ ]:
# City is now a database field
# def select_city(df,city="Vancouver"):
#     """Filters a dataframe to a city who's coordinates are defined in neighbourhoods"""
#     coords = cities[city]
#     df_filtered = df[df.apply(
#             lambda x: mplPath.Path(coords.as_matrix()).contains_point((x['longitude'],x['latitude'])),axis=1)]
#     return df_filtered

## Sanity Check
def check_data_for_wierdness(month):
    # Should really make some nice histograms here
    min_price = 500
    max_price = 7000
    print("Checking data stats for {}...".format(month))
    df = pd.read_sql_query("SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND city='Vancouver'".format(month),conn)
    print("{} Listings in Vancouver".format(df.shape[0]))
    print("{} of {} below ${}".format(df.loc[df['price'] < min_price].shape[0],df.shape[0],min_price))
    print("{} of {} above ${}".format(df.loc[df['price'] > max_price].shape[0],df.shape[0],max_price))
    print("{} of {} furnished. Price difference: ${}".format(df.loc[df['furnished'] == 1].shape[0],df.shape[0],df.loc[df['furnished'] == 1]['price'].median()-df['price'].median()))
    print("{} of {} missing # of bedrooms. Price difference: ${}".format(df.loc[df['bedrooms'].isnull()].shape[0],df.shape[0],df.loc[df['bedrooms'].isnull()]['price'].median()-df['price'].median()))
    
check_data_for_wierdness(months[-1])

In [ ]:
## Long Term Median Price Trend

def median_price_trend(months,online_plot=False):
    # Median Price of all listings for the month
    old_months = ['2016-02','2016-03','2016-04','2016-05','2016-06','2016-07','2016-08','2016-09','2016-10','2016-11','2016-12']

    month_nums = old_months+months
    # 2016-02 to 2016-12
    # Initializing with results from old data
    vancouver_prices = [1800,1700,1700,1800,1850,1950,2000,2100,2095,1995,1897]
    gvrd_prices = [1590,1500,1500,1650,1700,1800,1900,1900,1895,1850,1750]
    error_bar = [24.7,14.2,13.1,14.5,15.4,15.1,13.8,14.8,13.0,13.9,26.6]
    error_bar_gvrd = [15.9,9.3,9.0,9.5,9.9,9.9,9.9,9.4,9.7,8.9,9.3,18.4]
    
    van_month_nums = old_months.copy()
    gvrd_month_nums = old_months.copy()
    for month in months: # from 2017-01 and on
        # Vancouver Only
        sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND city='Vancouver'".format(month,min_price,max_price)
        data = pd.read_sql_query(sql,conn)
        van_med = data['price'].median()
        if not math.isnan(van_med):
            van_month_nums.append(month)
            vancouver_prices.append(van_med)
            error_bar.append(np.sqrt(np.pi/2)*data['price'].std()/np.sqrt(data.shape[0]))
            
        # Whole GVRD
        sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price != 'NULL' AND price BETWEEN {} AND {}".format(month,min_price,max_price)
        data = pd.read_sql_query(sql,conn)
        gvrd_med = data['price'].median()
        if not math.isnan(gvrd_med):
            gvrd_month_nums.append(month)
            gvrd_prices.append(gvrd_med)
            error_bar_gvrd.append(np.sqrt(np.pi/2)*data['price'].std()/np.sqrt(data.shape[0])) 
            
    plot = [Scatter(x=van_month_nums, 
                             y=vancouver_prices,
                             name="Median Rent in Vancouver",
                    error_y=dict(
                                type='data',
                                array=error_bar,
                                visible=True)
                             ),
            Scatter(x=gvrd_month_nums, 
                             y=gvrd_prices,
                             name="Median Rent in GVRD",
                                       error_y=dict(
                                type='data',
                                array=error_bar_gvrd,
                                visible=True
                             ))]
    layout = Layout(
        dict(title = 'Median Rent for Apartments',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Median Price ($)'),
              )
        )
    fig = dict(data=plot,layout=layout)
    if online_plot:
        py.plot(fig)
    else:
        iplot(fig)
    return list(zip(van_month_nums,vancouver_prices)), list(zip(gvrd_month_nums,gvrd_prices))

In [ ]:
def distribution_by_unit_type(df):
    # Plots a stacked histogram of the price distribution by unit type
    # fig = tools.make_subplots(rows=2, cols=1, subplot_titles=('Price', 'Price per sqft',))
    # subplots doesn't do the legend right
    price_plot = []
    ppsq_plot = []
    for utype in ['house','townhouse','condo','apartment']:
        df_type = df[df.unit_type == utype]
        df_type_w_area = df_type.loc[df_type['area'].notnull()]
        ppsq = (df_type_w_area['price']/df_type_w_area['area'])
        price_plot.append(Histogram(x=df_type['price'],name=utype,xbins=dict(start=500,end=7000,size=100)))
        ppsq_plot.append(Histogram(x=ppsq,name=utype,xbins=dict(start=0.5,end=5.0,size=0.1)))
    layout = dict(title='Price Distribution',barmode='stack')
    fig=dict(data=price_plot,layout=layout)
    iplot(fig)
    layout = dict(title='Price per Square Foot Distribution',barmode='stack')
    fig=dict(data=ppsq_plot,layout=layout)
    iplot(fig)

In [ ]:
## Median price broken down by bedroom
def trend_by_bedroom (months,online_plot=False,city="Vancouver"):
    # this function should produce the trend data for vancouver rentals by bedrooms
    # pass in a list of integers for months
#     all_months = ['January','Februrary','March','April','May','June','July','August','September','October', 'November', 'December']
    all_series = {}
    plots = []
    for bed in [0,1,2,3]:
        series = []
        error_bar = []
        for m in months:
            # Vancouver
            sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND furnished = 0".format(m,min_price,max_price)
            if city == "Vancouver":
                data = select_city(pd.read_sql_query(sql,conn))
            elif city == "GVRD":
                data = pd.read_sql_query(sql,conn)
            data = data[data.bedrooms == bed]
            series.append(data['price'].median())
            error_bar.append(np.sqrt(np.pi/2)*data['price'].std()/np.sqrt(data.shape[0]))
            
        all_series[str(bed) + ' Bedrooms'] = series
        plots.append(Scatter(x=months, 
                             y=series,
                             name=str(bed) + ' Bedrooms',
                             error_y=dict(
                                type='data',
                                array=error_bar,
                                visible=True)))
    # Plotting
    layout = Layout(
        dict(title = 'Median Rent in {} for Apartments'.format(city),
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Median Price ($)'),
              )
        )
    fig = Figure(data=plots,layout=layout)
    if online_plot:
        py.plot(fig)
    else:
        iplot(fig)
    #produce offline plot/image
    #plot(fig,filename='temp-plot.html',image='png',image_filename='Median Rent',image_width=1200, image_height=800)
    return all_series

trend_by_bedroom(months)

In [ ]:
## Rent by neighbourhood
def rent_by_neighbourhood (months,return_html=False):
    assert len(months) > 1
    data = []
    for hood in hoods.keys():
        # Latest Month
        sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND furnished = 0 AND neighbourhood = '{}'".format(months[-1],min_price,max_price,hood)
        df = pd.read_sql_query(sql,conn)
        
        # Previous Month(s)
        # Might want to move to a longer average here...
        sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND furnished = 0 AND  neighbourhood = '{}'".format(months[-2],min_price,max_price,hood)
        prev_df = pd.read_sql_query(sql,conn)
        med_price = hood,df['price'].median()
        med_area = df['area'].median()
        ppsq = (df['price']/df['area']).median()
        prev_ppsq = (prev_df['price']/prev_df['area']).median()
        # should divide first, then calculate the median!
        data.append([hood,"${}".format(df['price'].median()),df['area'].median(),"${:0.2f}".format(ppsq),"{:.0%}".format((ppsq-prev_ppsq)/prev_ppsq)])
    if return_html==True:
        return tabulate.tabulate(data, headers=["Neighbourhood","Median Rent","Median Area","Rent per square foot"],tablefmt='html')
    else:
        display(HTML(tabulate.tabulate(data, headers=["Neighbourhood","Median Rent","Median Area","Rent per square foot","Change in Price per square foot(%)"],tablefmt='html')))
#rent_by_neighbourhood(months)

In [ ]:
def rent_vs_area(month):
    # plots a rent-vs-area regression. Linear is not a good model for prices, in general
    sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND furnished = 0 AND area BETWEEN {} AND {}".format(month,min_price,max_price,100,15000)
    df = select_city(pd.read_sql_query(sql,conn))
    regr = linear_model.LinearRegression()
    regr.fit(df['area'].reshape(-1,1), df['price'].reshape(-1,1))
    plot = Scatter(
        x = df['area'],
        y = df['price'],
        mode = 'markers')
    print('Coefficients: \n', regr.coef_)
    predict = Scatter(
        x = df['area'],
        y = regr.predict(df['area'].reshape(-1,1)).flatten(),
        mode = 'lines')
    layout = Layout(
        dict(title = 'Price vs area',
              xaxis = dict(title = 'Area (sqft)'),
              yaxis = dict(title = 'Price ($)'),
              )
        )
    fig = Figure(data = [plot,predict],layout = layout)
    iplot(fig)


In [8]:
def median_rent_psf (dataframe):
    # returns rent per sqft for the given dataset
    # first, filter listings to those with reasonable areas listed
    df = dataframe
    
    # filter to reasonable area and price
    df = df[df['area'] > 250] 
    df = df[df['area'] < 10000]
    df = df[df['price'] > 250]
    df = df[df['price'] < 7000]
    df = df[df['furnished'] == 0]
    
    rent_psf = df['price']/df['area']
    return rent_psf.median()
    
    

def listings_for_month(month):
    # unfurnished listings filtered for outliers in price
    sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}'".format(month)
    df = pd.read_sql_query(sql,conn)
    return df


df = listings_for_month('2018-11')
median_rent_psf(df)

2.1538461538461537

In [ ]:
## Maping
mapbox_access_token = 'pk.eyJ1IjoiYnJhbWV2ZXJ0IiwiYSI6ImNqOHk0aHMxOTI4eDUzNHF6dDBiMmNvbjkifQ.mqP1zmto8tR9O6yTOaeP7w'

def generate_plain_markers(df,series_name):
    info = df.apply(lambda row: unit_desc(row), axis=1).values.tolist()
    plot = (dict(type = 'scattermapbox',
                lat=df.latitude, 
                lon=df.longitude, 
                name=series_name,
                hoverinfo = 'text',
                text = info,
                mode = 'markers'))
    return plot
    
def generate_price_markers(df,series_name):
    prices = df.price
    info = df.apply(lambda row: unit_desc(row), axis=1).values.tolist()
    colorscale = list(zip(np.linspace(0.0,1.0,num=11),cl.scales['11']['div']['RdYlBu'][::-1]))
    plot = (dict(type = 'scattermapbox',
                lat=df.latitude, 
                lon=df.longitude, 
                name=series_name,
                hoverinfo = 'text',
                text = info,
                mode = 'markers',
                marker = Marker(color=df.price, colorscale=colorscale,cmax=3500,cmin=750,autocolorscale=False,cauto = False)))
    return plot

def generate_ppsf_markers(df,series_name):
    df = df.loc[df['area'].notnull()]
    ppsf = df['price']/df['area']
    info = df.apply(lambda row: unit_desc(row), axis=1).values.tolist()
    colorscale = list(zip(np.linspace(0.0,1.0,num=11),cl.scales['11']['div']['RdYlBu'][::-1]))
    plot = (dict(type = 'scattermapbox',
                lat=df.latitude, 
                lon=df.longitude, 
                name=series_name,
                hoverinfo = 'text',
                text = info,
                mode = 'markers',
                marker = Marker(color=ppsf, colorscale=colorscale,cmax=5.0,cmin=1.0,autocolorscale=False,cauto = False,colorbar=ColorBar(title='$-per-square-foot'))))
    return plot

def map_by_bedroom(month):
    # maps prices broken out by bedroom
    sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND furnished = 0".format(month,min_price,max_price)
    df = select_city(pd.read_sql_query(sql,conn))
    # This function takes a dataframe and plots on the Vancouver map
    plots = []
    for bed in [0,1,2,3]:
        df_bed = df[df.bedrooms == bed]
        plots.append(generate_markers(df_bed,str(bed)+' Bedrooms'))
    layout = Layout(
            autosize=True,
            mapbox = dict(
                accesstoken=mapbox_access_token,
                domain = dict(x=[0,1],y=[0,1]),
                center = dict(lon=-123.1,lat=49.26),
                zoom = 10.5))
    fig = Figure(data=plots,layout=layout)
    py.iplot(fig,filename="bedroom-"+month)
    
def map_by_type(month):
    # maps listing by housing type
    sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND furnished = 0".format(month,min_price,max_price)
    df = select_city(pd.read_sql_query(sql,conn))
    # This function takes a dataframe and plots on the Vancouver map
    plots = []
    for utype in ['house','condo','townhouse','apartment']:
        df_type = df[df.unit_type == utype]
        plots.append(generate_plain_markers(df_type,utype))
    layout = Layout(
            autosize=True,
            mapbox = dict(
                accesstoken=mapbox_access_token,
                domain = dict(x=[0,1],y=[0,1]),
                center = dict(lon=-123.1,lat=49.26),
                zoom = 10.5))
    fig = Figure(data=plots,layout=layout)
    py.iplot(fig,filename="housing-type-"+month)
    
def map_all(month):
    # maps listing with price
    sql = "SELECT * FROM apartments WHERE strftime(\"%Y-%m\", date) = '{}' AND price BETWEEN {} AND {} AND furnished = 0".format(month,min_price,max_price)
    df = pd.read_sql_query(sql,conn)
    # This function takes a dataframe and plots on the Vancouver map
    plots = [(generate_ppsf_markers(df,'Rent map '+month))]
    layout = Layout(
            autosize=True,
            mapbox = dict(
                accesstoken=mapbox_access_token,
                domain = dict(x=[0,1],y=[0,1]),
                center = dict(lon=-123.1,lat=49.26),
                zoom = 10.5))
    fig = Figure(data=plots,layout=layout)
    py.iplot(fig,filename='Rent map '+month)

def unit_desc(row):
    if (not np.isnan(row['bedrooms'])) and (row['unit_type'] != None):
        return "{} bedroom {}, ${}".format(int(row['bedrooms']),row['unit_type'], row['price'])
    elif (not np.isnan(row['bedrooms'])):
        return "{} bedroom, ${}".format(int(row['bedrooms']), row['price'])
    else:
        return "${}".format(row['price'])

In [ ]:
# Report
vancouver_median_price, gvrd_median_price = median_price_trend(months)